<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>

<h1 align="center"><font size="5">Building Regression Model In Keras Assignment Part D</font></h1>
<h1 align="center"><font size="3">Mridul Sharma</font></h1>


Let's first load required libraries:

In [1]:
#importing necessary packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
#notice: Disable all warnings 
import warnings
warnings.filterwarnings('ignore')

Let's load the dataset

In [3]:
!wget -O concrete_data.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv

--2022-11-07 21:03:28--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58988 (58K) [text/csv]
Saving to: ‘concrete_data.csv’

concrete_data.csv   100%[===================>]  57.61K   333KB/s    in 0.2s    

2022-11-07 21:03:29 (333 KB/s) - ‘concrete_data.csv’ saved [58988/58988]



### Load Data from CSV file

In [4]:
#read data as pandas dataframe
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:

1. Cement

2. Blast Furnace Slag

3. Fly Ash

4. Water

5. Superplasticizer

6. Coarse Aggregate

7. Fine Aggregate

By Observing, we can say that the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.


Let's take a look at our data:

In [5]:
#Describe data
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
#Shape of the dataframe in (rows, columns)
concrete_data.shape

(1030, 9)

Checking for the null values

In [7]:
#Checking the missing values
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

We have no missing data and the dataset looks pretty clean. So, we can use this data for training/testing purpose to build our model.

Let's separate features and target variables.

In [8]:
#Separaring Features and Target in to X and y
concrete_data_columns = concrete_data.columns
X = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
y= concrete_data['Strength'] # Strength column

In [9]:
#Calculate number of predictors
n_predictors = X.shape[1]
n_predictors

8

### Normalizing Data

One way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation. We can achieve that by using StandardScaler from scikit-learn library.

In [10]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
scaler_y = StandardScaler()

In [11]:
scaler_x.fit(X.values)

StandardScaler()

In [12]:
X = scaler_x.transform(X.values)

In [13]:
scaler_y.fit(y.values.reshape(-1, 1))

StandardScaler()

In [14]:
y = scaler_y.transform(y.values.reshape(-1, 1))

### Building a Neural Network

In [15]:
#importing Keras
import keras

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [16]:
from keras.models import Sequential
from keras.layers import Dense

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [17]:
# define regression model
def keras_regression_model():
    # create model
    model = Sequential()
    model.add(Dense(7, input_dim = n_predictors, activation = 'relu', kernel_initializer = 'normal'))
    model.add(Dense(10, activation='relu', input_shape=(n_predictors,)))
    model.add(Dense(10, activation = 'relu'))
    model.add(Dense(10, activation = 'relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [18]:
#Calling function to create our linear regression model
model = keras_regression_model()

Splitting the training and testing set:

In [19]:
X.shape

(1030, 8)

In [20]:
y.shape

(1030, 1)

In [21]:
#import train_test_split
from sklearn.model_selection import train_test_split

# Split data 70%-30% into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

print ('Training Set: %d rows\nTest Set: %d rows' % (X_train.shape[0], X_test.shape[0]))

Training Set: 721 rows
Test Set: 309 rows


### Train and Test our model

In [22]:
#fit the model
model.fit(X_train, y_train, epochs = 50, batch_size = 1, verbose = 1)

Epoch 1/50
721/721 [==============================] - 2s 3ms/step - loss: 0.5338
Epoch 2/50
721/721 [==============================] - 2s 2ms/step - loss: 0.2522
Epoch 3/50
721/721 [==============================] - 1s 2ms/step - loss: 0.1985
Epoch 4/50
721/721 [==============================] - 2s 2ms/step - loss: 0.1699
Epoch 5/50
721/721 [==============================] - 2s 2ms/step - loss: 0.1664
Epoch 6/50
721/721 [==============================] - 2s 2ms/step - loss: 0.1595
Epoch 7/50
721/721 [==============================] - 2s 2ms/step - loss: 0.1466
Epoch 8/50
721/721 [==============================] - 2s 2ms/step - loss: 0.1451
Epoch 9/50
721/721 [==============================] - 2s 2ms/step - loss: 0.1411
Epoch 10/50
721/721 [==============================] - 2s 2ms/step - loss: 0.1326
Epoch 11/50
721/721 [==============================] - 2s 2ms/step - loss: 0.1283
Epoch 12/50
721/721 [==============================] - 2s 2ms/step - loss: 0.1296
Epoch 13/50
721/721 [====

Now, we need to compute the mean squared error between the predicted concrete strength and actual concrete strength.

In [23]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

10/10 [==============================] - 0s 2ms/step - loss: 0.1456


0.14560329914093018

Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the mean_squared_error function from Scikit-learn.

In [24]:
from sklearn.metrics import mean_squared_error

In [25]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

0.1456033092926455 0.0


Create a list of 50 mean squared errors and report mean and the standard deviation of the mean squared errors.

In [26]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. "+"\n"+ "Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 0.10062108188867569
MSE 2: 0.11257573217153549
MSE 3: 0.07702350616455078
MSE 4: 0.08802471309900284
MSE 5: 0.0941634327173233
MSE 6: 0.0966593399643898
MSE 7: 0.10755651444196701
MSE 8: 0.07196944952011108
MSE 9: 0.07923054695129395
MSE 10: 0.06991398334503174
MSE 11: 0.07195831090211868
MSE 12: 0.07529062032699585
MSE 13: 0.0940597653388977
MSE 14: 0.09223834425210953
MSE 15: 0.07729824632406235
MSE 16: 0.06117308884859085
MSE 17: 0.06001261994242668
MSE 18: 0.06274525076150894
MSE 19: 0.06298013776540756
MSE 20: 0.06878606230020523
MSE 21: 0.059432025998830795
MSE 22: 0.0641729086637497
MSE 23: 0.05854694917798042
MSE 24: 0.06273315101861954
MSE 25: 0.05820576101541519
MSE 26: 0.0705837830901146
MSE 27: 0.06817533820867538
MSE 28: 0.054203372448682785
MSE 29: 0.06933696568012238
MSE 30: 0.061091069132089615
MSE 31: 0.059024155139923096
MSE 32: 0.05099138244986534
MSE 33: 0.04730367660522461
MSE 34: 0.06443049013614655
MSE 35: 0.054923512041568756
MSE 36: 0.06770754605531693
M

Mean and Standard Deviation of MSE is a little bit better when we use three hidden layer instead of a single hidden layer.